In [15]:
import numpy as np
from scipy import io
from libs.utils import read_int16_as_float32, write__float32_as_int16
%matplotlib inline
import matplotlib.pyplot as plt
from libs.utils import down_sampling
import IPython.display

In [16]:
def conv_imp(target_wav, chs=[3,4], degree='270'):
    impfile = 'MIRD/Impulse_response_Acoustic_Lab_Bar-Ilan_University_(Reverberation_0.160s)_3-3-3-8-3-3-3_1m_{}.mat'.format(degree)
    matdata = io.loadmat(impfile,squeeze_me=True)
    imp_mat = matdata["impulse_response"]
    imps = [down_sampling(imp_mat[200:7000,ch], 48000, 16000) for ch in chs]
    #imps = [imp_mat[200:10000,ch] for ch in chs]
    #plt.figure();plt.plot(imps[0])
    target_wavs=[]
    for imp, ch in zip(imps, chs):
        target_wavs.append(np.convolve(target_wav, imp))
    return np.array(target_wavs)


def make_noisy(angles, snr):
    noise_wav = read_int16_as_float32('MIRD/{}.wav'.format('09'), 16000)
    clean_wav = read_int16_as_float32('MIRD/nf001001.wav', 16000)
    
    clean_multi = conv_imp(clean_wav, chs=[3,4], degree='000')
    noises_multi = np.zeros(clean_multi.shape)
    for angle in angles:
        noise_multi = conv_imp(noise_wav, chs=[3,4], degree='{:03}'.format(360-angle))
        noise_multi = noise_multi[:,:clean_multi.shape[1]]
        noises_multi = noises_multi + noise_multi
    coeff = calc_noise_coeff(noises_multi[0,:], clean_multi[0,:], snr)
    noisy_multi = clean_multi + noises_multi * coeff
    return noisy_multi

In [ ]:
make_noisy(30, -10)